## Импорты и данные

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

In [2]:
df = pd.read_csv('synthetic_gmv_data_1.2.csv')
df

,user_id,gmv,group_name
0,myo4ixol31,1428,test
1,myo4ixol31,1428,test
2,myo4ixol31,1071,test
3,myo4ixol31,1071,test
4,pkzf2889ww,351,test
...,...,...,...
799056,egoeez9fs9,712,control
799057,egoeez9fs9,641,control
799058,egoeez9fs9,855,control
799059,da9kujvymq,1110,control


In [3]:
data = df.groupby(['user_id', 'group_name'], as_index=False).agg({'gmv': 'sum', 'user_id':'count'})
data

,group_name,gmv,user_id
0,control,733,1
1,test,3187,3
2,control,2933,6
3,test,1695,5
4,control,1496,4
...,...,...,...
196386,test,1141,2
196387,control,2364,4
196388,test,1923,4
196389,control,1235,2


In [4]:
a = data.loc[data.group_name == 'control']
b = data.loc[data.group_name == 'test']

## Расчёт дисперсии дельта-методом

In [5]:
def safe_divide(x, y):
    try:
        return x / y
    except ZeroDivisionError:
        return np.nan

In [7]:

def delta_var(numerator, denominator):
    x = numerator
    y = denominator
    n = len(x)
    mu_x = np.mean(x)
    mu_y = np.mean(y)
    var_x = np.var(x, ddof=1)
    var_y = np.var(y, ddof=1)
    cov_xy = np.cov(x, y, ddof=1)[0][1]    
    delta_var = safe_divide(safe_divide(var_x, mu_y**2)  - 2*cov_xy*safe_divide(mu_x, mu_y**3) + var_y*safe_divide(mu_x**2,mu_y**4), n)
    return delta_var

In [ ]:
n = a.gmv.shape[0]
m = b.gmv.shape[0]

a_var = delta_var(a.gmv, a.user_id)
b_var = delta_var(b.gmv, b.user_id)
sigma = np.sqrt(a_var + b_var)
delta_estimator = safe_divide(np.mean(a.gmv), np.mean(a.user_id)) - safe_divide(np.mean(b.gmv), np.mean(b.user_id))

tt = safe_divide(delta_estimator, sigma)
pvalue = 2*stats.t.sf(np.abs(tt), n+m-2)

In [9]:
round(tt, 3), round(pvalue, 3)

(-2.344, 0.019)